In [2]:
# install our package
! pip install .

Processing /Users/aidensmac/Documents/STA663/663-Final-Project
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/47/9666xjq91wggdq4vpz3q_r_80000gn/T/pip-req-build-y80ad3vm/setup.py'"'"'; __file__='"'"'/private/var/folders/47/9666xjq91wggdq4vpz3q_r_80000gn/T/pip-req-build-y80ad3vm/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/47/9666xjq91wggdq4vpz3q_r_80000gn/T/pip-wheel-y3a_80sg
       cwd: /private/var/folders/47/9666xjq91wggdq4vpz3q_r_80000gn/T/pip-req-build-y80ad3vm/
  Complete output (23 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.7
  creating build/lib.macosx-10.9-x86_64-3.7/Sinkhorn_663
  copying Sinkhorn_663/Sinkhorn_main.

In [74]:
from Sinkhorn_663 import Sinkhorn, log_domain_sinkhorn_2, Sinkhorn_numba, sample_to_prob_vec, sample_to_prob_vec_nD
from Skh_cpp import Sinkhorn_cpp

## Test

### Simulation Data

In [75]:
import numpy as np

In [76]:
# create simulation data
N = 5000
np.random.seed(1)
u = np.random.normal(loc = 0, size = N)
v = np.random.normal(loc = 10, size = N)
Cost, p, q = sample_to_prob_vec(u, v, 1)

### Check results

In [77]:
# Serial Numba version
Sinkhorn(p, q, Cost)

(9.96780555006323, 3)

In [78]:
# Parallel Numba version in log domain
Sinkhorn(p, q, Cost, log_domain = True)

array([9.96780555])

In [73]:
# cpp version
Sinkhorn_cpp(p, q.reshape(-1, 1), Cost, 20, 1e-6, 50)

array([9.96780555])

## Time

In [79]:
%timeit -r3 -n3 Sinkhorn(p, q, Cost)

4.52 s ± 140 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [80]:
%timeit -r3 -n3 Sinkhorn(p, q, Cost, log_domain = True)

2.53 s ± 123 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [81]:
%timeit -r3 -n3 Sinkhorn_cpp(p, q.reshape(-1, 1), Cost, 20, 1e-6, 50)

4.34 s ± 262 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
